# Embedding XLNET

**摘要：首先呢，需要承认一件事情，就是我还没有对XLNET去很好的了解=====。XLNET也是基于transfromer网络的，是google在推出BERT后再推出的网络，性能比BERT会更好。由于XLNET也是语言模型，所以也是可以做文本向量化的,本文采用XLNET中文预训练模型(chinese_xlnet_mid_L-24_H-768_A-12)进行向量化**

[参考源码地址==========](https://github.com/yongzhuo/Keras-TextClassification)

- 预训练模型说明
    - 来源：哈工大进行中文预训练的
    - [github原地址======](https://github.com/ymcui/Chinese-PreTrained-XLNet)
    - 网络结构：XLNet-mid：24-layer, 768-hidden, 12-heads, 209M parameters
    - 下载tensorflow版本(具体请打开链接查看)
    
XLNET以后会去探索，暂时呢就不知道了-------
不过呢，不知道也可以先用着嘛=======【大笑】

这里记录下今天（2019-11-27）探索BXLNET Embedding的实现代码，以便于以后有需要的时候能快速实现。

In [3]:
from keras.layers import Add, Embedding
from gensim.models import KeyedVectors
from keras.models import Input, Model

import numpy as np
import codecs
import os
import re

path_embedding_xlnet = "/Users/zhouwencheng/Desktop/Grass/data/model" \
                       "/ImportModel/XLNET/chinese_xlnet_mid_L-24_H-768_A-12"


In [4]:
from __future__ import print_function, division
from keras.engine import Layer


class NonMaskingLayer(Layer):
    """
    fix convolutional 1D can't receive masked input, detail: https://github.com/keras-team/keras/issues/4978
    thanks for https://github.com/jacoxu
    """

    def __init__(self, **kwargs):
        self.supports_masking = True
        super(NonMaskingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        pass

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        return x

    def compute_output_shape(self, input_shape):
        return input_shape

In [10]:
class XlnetEmbedding(object):
    def __init__(self,
                 len_max=50,  # 文本最大长度, 建议25-50
                 embed_size=300,  # 嵌入层尺寸
                 vocab_size=30000,  # 字典大小, 这里随便填的，会根据代码里修改
                 trainable=False,  # 是否训练参数
                 path_mode=path_embedding_xlnet,
                 layer_indexes=[24], # 默认取最后一层的输出 大于13则取最后一层的输出
                 xlnet_embed={},
                 batch_size=2
                ):
        self.len_max = len_max
        self.embed_size = embed_size
        self.vocab_size = vocab_size
        self.trainable = trainable 
        self.corpus_path = path_mode
        self.layer_indexes = layer_indexes
        self.xlnet_embed = xlnet_embed
        self.batch_size = batch_size
        
        self.input = None
        self.output = None
        self.model = None 
        self.build()
        
 
    def build(self):
        from keras_xlnet import Tokenizer, ATTENTION_TYPE_BI, ATTENTION_TYPE_UNI
        from keras_xlnet import load_trained_model_from_checkpoint

        self.checkpoint_path = os.path.join(self.corpus_path, 'xlnet_model.ckpt')
        self.config_path = os.path.join(self.corpus_path, 'xlnet_config.json')
        self.spiece_model = os.path.join(self.corpus_path, 'spiece.model')

        self.attention_type = 'bi'
        self.attention_type = ATTENTION_TYPE_BI if self.attention_type == 'bi' else ATTENTION_TYPE_UNI
        self.memory_len = 0
        self.target_len = 5
        print('load xlnet model start!')
        model = load_trained_model_from_checkpoint(checkpoint_path=self.checkpoint_path,
                                                   attention_type=self.attention_type,
                                                   in_train_phase=self.trainable,
                                                   config_path=self.config_path,
                                                   memory_len=self.memory_len,
                                                   target_len=self.target_len,
                                                   batch_size=self.batch_size,
                                                   mask_index=0)
#         model.summary()
        print('load xlnet model finish!')
        # 字典加载
        self.tokenizer = Tokenizer(self.spiece_model)
        self.model_layers = model.layers
        len_layers = self.model_layers.__len__()
        print(len_layers)
        len_couche = int((len_layers - 6) / 10)
        # 一共246个layer
        # 每层10个layer（MultiHeadAttention,Dropout,Add,LayerNormalization）,第一是9个layer的输入和embedding层
        # 一共24层
        layer_dict = [5]
        layer_0 = 6
        for i in range(len_couche):
            layer_0 = layer_0 + 10
            layer_dict.append(layer_0-2)
        # 输出它本身
        if len(self.layer_indexes) == 0:
            encoder_layer = model.output
            # 分类如果只有一层，取得不正确的话就取倒数第二层
        elif len(self.layer_indexes) == 1:
            if self.layer_indexes[0] in [i + 1 for i in range(len_couche + 1)]:
                encoder_layer = model.get_layer(index=layer_dict[self.layer_indexes[0]]).output
            else:
                encoder_layer = model.get_layer(index=layer_dict[-1]).output
        # 否则遍历需要取的层，把所有层的weight取出来并加起来shape:768*层数
        else:
            # layer_indexes must be [0, 1, 2,3,......24]
            all_layers = [model.get_layer(index=layer_dict[lay]).output
                          if lay in [i + 1 for i in range(len_couche + 1)]
                          else model.get_layer(index=layer_dict[-1]).output  # 如果给出不正确，就默认输出倒数第一层
                          for lay in self.layer_indexes]
            print(self.layer_indexes)
            print(all_layers)
            all_layers_select = []
            for all_layers_one in all_layers:
                all_layers_select.append(all_layers_one)
            encoder_layer = Add()(all_layers_select)
            print(encoder_layer.shape)
        self.output = NonMaskingLayer()(encoder_layer)
        self.input = model.inputs
        self.model = Model(inputs=self.input, outputs=self.output)

        self.embedding_size = self.model.output_shape[-1]
        self.vocab_size = len(self.tokenizer.sp)

    def sentence2idx(self, text):
        text = self.extract_chinese(str(text).upper())
        tokens = self.tokenizer.encode(text)
        tokens = tokens + [0] * (self.target_len - len(tokens)) \
            if len(tokens) < self.target_len \
            else tokens[0:self.target_len]
        token_input = np.expand_dims(np.array(tokens), axis=0)
        segment_input = np.zeros_like(token_input)
        memory_length_input = np.zeros((1, 1))
        return [token_input, segment_input, memory_length_input]
    
    
    def extract_chinese(self, text):
        """
          只提取出中文、字母和数字
        :param text: str, input of sentence
        :return:
        """
        chinese_exttract = ''.join(re.findall(u"([\u4e00-\u9fa5A-Za-z0-9@._])", text))
        return chinese_exttract   

In [11]:
texts = ["今天天气不错",
             "明天天气也不错"]
eb = XlnetEmbedding()
x = []
x_segment = []
x_memory = []
for t in texts:
    x_buff, x_segment_buff, x_memory_buff = eb.sentence2idx(t)
    x.append(x_buff[0])
    x_segment.append(x_segment_buff[0])
    x_memory.append(x_memory_buff[0])

x = np.array(x)
x_segment = np.array(x_segment)
x_memory = np.array(x_memory)

print(x.shape)
print(x_segment.shape)
print(x_memory.shape)

print(x)
print(x_segment)
print(x_memory)

model = eb.model
p = model.predict([x, x_segment, x_memory])
print(p.shape)
print(p)
print(p.shape)

load xlnet model start!





load xlnet model finish!
126
(2, 5)
(2, 5)
(2, 1)
[[   19  4328 10022    63  4856]
 [ 7350   118 10022  4180  4856]]
[[0 0 0 0 0]
 [0 0 0 0 0]]
[[0.]
 [0.]]
(2, 5, 768)
[[[-0.01283701  1.0316377   0.6240789  ... -0.25227237 -0.38330236
   -0.26968622]
  [ 0.11844011  1.0428874  -0.1372549  ... -0.47099146 -0.16267313
   -0.01897161]
  [ 0.7833793   0.7908515  -0.07405517 ... -0.11398397 -0.38520646
    0.34588325]
  [ 0.9593434   0.8058133   0.6556412  ... -0.4739001  -0.13973673
    0.4029246 ]
  [ 0.33478862  0.13342354  1.1380457  ... -1.1246004   1.0664203
    0.34830493]]

 [[-0.8383907   1.1733837   0.07351154 ... -0.7073219  -0.21703273
    0.32910395]
  [-0.8203141   1.302698   -0.3939988  ... -0.6257708  -0.29780596
    0.27401137]
  [-0.6780075   0.5064006  -0.47468746 ... -0.31255782 -0.42654335
    0.7236039 ]
  [-0.6743382   0.04179962  0.9269874  ... -0.5992737  -0.8316488
    1.5014975 ]
  [-0.23544422 -0.9687195   1.2791495  ... -1.1714749  